In [1]:
import cv2
import numpy as np

import matplotlib.pyplot as plt

In [2]:
import dlib
dlib.__version__

'19.9.0'

In [3]:
import numpy
numpy.__version__

'1.11.3'

In [4]:
import pandas
pandas.__version__

'0.23.4'

In [8]:
path = r"F:\PYcode\coding\faceinfo\007.jpg"
img = cv2.imread(path)

In [5]:
win = dlib.image_window()

In [9]:
# The contents of this file are in the public domain. See LICENSE_FOR_EXAMPLE_PROGRAMS.txt
#
#   This example shows how to run a CNN based face detector using dlib.  The
#   example loads a pretrained model and uses it to find faces in images.  The
#   CNN model is much more accurate than the HOG based model shown in the
#   face_detector.py example, but takes much more computational power to
#   run, and is meant to be executed on a GPU to attain reasonable speed.
#
#   You can download the pre-trained model from:
#       http://dlib.net/files/mmod_human_face_detector.dat.bz2
#
#   The examples/faces folder contains some jpg images of people.  You can run
#   this program on them and see the detections by executing the
#   following command:
#       ./cnn_face_detector.py mmod_human_face_detector.dat ../examples/faces/*.jpg
#
#
# COMPILING/INSTALLING THE DLIB PYTHON INTERFACE
#   You can install dlib using the command:
#       pip install dlib
#
#   Alternatively, if you want to compile dlib yourself then go into the dlib
#   root folder and run:
#       python setup.py install
#
#   Compiling dlib should work on any operating system so long as you have
#   CMake installed.  On Ubuntu, this can be done easily by running the
#   command:
#       sudo apt-get install cmake
#
#   Also note that this example requires Numpy which can be installed
#   via the command:
#       pip install numpy
import sys
import dlib
import cv2

# if len(sys.argv) < 3:
#     print(
#         "Call this program like this:\n"
#         "   ./cnn_face_detector.py mmod_human_face_detector.dat ../examples/faces/*.jpg\n"
#         "You can get the mmod_human_face_detector.dat file from:\n"
#         "    http://dlib.net/files/mmod_human_face_detector.dat.bz2")
#     exit()

model_url = "F:/PYcode/coding/faceinfo/dlib_shapedata/mmod_human_face_detector.dat"
cnn_face_detector = dlib.cnn_face_detection_model_v1(model_url)
win = dlib.image_window()

img_url = "F:/PYcode/coding/faceinfo/infomations/007.jpg"

print("Processing file: {}".format(img_url))
img = cv2.imread(img_url, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# The 1 in the second argument indicates that we should upsample the image
# 1 time.  This will make everything bigger and allow us to detect more
# faces.
dets = cnn_face_detector(img, 1)
'''
This detector returns a mmod_rectangles object. This object contains a list of mmod_rectangle objects.
These objects can be accessed by simply iterating over the mmod_rectangles object
The mmod_rectangle object has two member variables, a dlib.rectangle object, and a confidence score.

It is also possible to pass a list of images to the detector.
    - like this: dets = cnn_face_detector([image list], upsample_num, batch_size = 128)

In this case it will return a mmod_rectangless object.
This object behaves just like a list of lists and can be iterated over.
'''
print("Number of faces detected: {}".format(len(dets)))
for i, d in enumerate(dets):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {} Confidence: {}".format(
        i, d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom(), d.confidence))

rects = dlib.rectangles()
rects.extend([d.rect for d in dets])

win.clear_overlay()
win.set_image(img)
win.add_overlay(rects)
dlib.hit_enter_to_continue()

Processing file: F:/PYcode/coding/faceinfo/infomations/007.jpg
Number of faces detected: 1
Detection 0: Left: 171 Top: 206 Right: 416 Bottom: 450 Confidence: 1.0425950288772583


In [ ]:
import sys
import os
import dlib
import glob
import cv2

# if len(sys.argv) != 5:
#     print(
#         "Call this program like this:\n"
#         "   ./face_clustering.py shape_predictor_5_face_landmarks.dat dlib_face_recognition_resnet_model_v1.dat ../examples/faces output_folder\n"
#         "You can download a trained facial shape predictor and recognition model from:\n"
#         "    http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2\n"
#         "    http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2")
#     exit()

predictor_path = r"F:/PYcode/coding/faceinfo/dlib_shapedata/shape_predictor_5_face_landmarks.dat"
face_rec_model_path = r"F:/PYcode/coding/faceinfo/dlib_shapedata/dlib_face_recognition_resnet_model_v1.dat"
faces_folder_path = r"F:/PYcode/coding/faceinfo/infomations"
output_folder_path = r"F:/PYcode/coding/faceinfo/infomations"

# Load all the models we need: a detector to find the faces, a shape predictor
# to find face landmarks so we can precisely localize the face, and finally the
# face recognition model.
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

descriptors = []
images = []

# Now find all the faces and compute 128D face descriptors for each face.
for f in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
    print("Processing file: {}".format(f))
#     img = dlib.load_rgb_image(f)
    img = cv2.imread(f, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(img, 1)
    print("Number of faces detected: {}".format(len(dets)))

    # Now process each face we found.
    for k, d in enumerate(dets):
        # Get the landmarks/parts for the face in box d.
        shape = sp(img, d)

        # Compute the 128D vector that describes the face in img identified by
        # shape.  
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        descriptors.append(face_descriptor)
        images.append((img, shape))

# Now let's cluster the faces.  
labels = dlib.chinese_whispers_clustering(descriptors, 0.5)
num_classes = len(set(labels))
print("Number of clusters: {}".format(num_classes))

# Find biggest class
biggest_class = None
biggest_class_length = 0
for i in range(0, num_classes):
    class_length = len([label for label in labels if label == i])
    if class_length > biggest_class_length:
        biggest_class_length = class_length
        biggest_class = i

print("Biggest cluster id number: {}".format(biggest_class))
print("Number of faces in biggest cluster: {}".format(biggest_class_length))

# Find the indices for the biggest class
indices = []
for i, label in enumerate(labels):
    if label == biggest_class:
        indices.append(i)

print("Indices of images in the biggest cluster: {}".format(str(indices)))

# Ensure output directory exists
if not os.path.isdir(output_folder_path):
    os.makedirs(output_folder_path)

# Save the extracted faces
print("Saving faces in largest cluster to output folder...")
for i, index in enumerate(indices):
    img, shape = images[index]
    file_path = os.path.join(output_folder_path, "face_" + str(i))
    # The size and padding arguments are optional with default size=150x150 and padding=0.25
    dlib.save_face_chip(img, shape, file_path, size=150, padding=0.25)
    

In [ ]:
import sys

import dlib

detector = dlib.get_frontal_face_detector()
win = dlib.image_window()

for f in sys.argv[1:]:
    print("Processing file: {}".format(f))
    img = dlib.load_rgb_image(f)
    # The 1 in the second argument indicates that we should upsample the image
    # 1 time.  This will make everything bigger and allow us to detect more
    # faces.
    dets = detector(img, 1)
    print("Number of faces detected: {}".format(len(dets)))
    for i, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            i, d.left(), d.top(), d.right(), d.bottom()))

    win.clear_overlay()
    win.set_image(img)
    win.add_overlay(dets)
    dlib.hit_enter_to_continue()


# Finally, if you really want to you can ask the detector to tell you the score
# for each detection.  The score is bigger for more confident detections.
# The third argument to run is an optional adjustment to the detection threshold,
# where a negative value will return more detections and a positive value fewer.
# Also, the idx tells you which of the face sub-detectors matched.  This can be
# used to broadly identify faces in different orientations.
if (len(sys.argv[1:]) > 0):
    img = dlib.load_rgb_image(sys.argv[1])
    dets, scores, idx = detector.run(img, 1, -1)
    for i, d in enumerate(dets):
        print("Detection {}, score: {}, face_type:{}".format(
            d, scores[i], idx[i]))

In [10]:
import sys
import os
import dlib
import glob
import cv2

# if len(sys.argv) != 5:
#     print(
#         "Call this program like this:\n"
#         "   ./face_clustering.py shape_predictor_5_face_landmarks.dat dlib_face_recognition_resnet_model_v1.dat ../examples/faces output_folder\n"
#         "You can download a trained facial shape predictor and recognition model from:\n"
#         "    http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2\n"
#         "    http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2")
#     exit()

predictor_path = r"F:/PYcode/coding/faceinfo/dlib_shapedata/shape_predictor_5_face_landmarks.dat"
face_rec_model_path = r"F:/PYcode/coding/faceinfo/dlib_shapedata/dlib_face_recognition_resnet_model_v1.dat"
faces_folder_path = r"F:/PYcode/coding/faceinfo/infomations"
output_folder_path = r"F:/PYcode/coding/faceinfo/infomations/out"

# Load all the models we need: a detector to find the faces, a shape predictor
# to find face landmarks so we can precisely localize the face, and finally the
# face recognition model.
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

descriptors = []
images = []

# Now find all the faces and compute 128D face descriptors for each face.
for f in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
    print("Processing file: {}".format(f))
    img = cv2.imread(f, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(img, 1)
    print("Number of faces detected: {}".format(len(dets)))

    # Now process each face we found.
    for k, d in enumerate(dets):
        # Get the landmarks/parts for the face in box d.
        shape = sp(img, d)

        # Compute the 128D vector that describes the face in img identified by
        # shape.  
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        descriptors.append(face_descriptor)
        images.append((img, shape))
# print(descriptors[0])
print(len(descriptors))
# Now let's cluster the faces.  
labels = dlib.chinese_whispers_clustering(descriptors, 0.4)
num_classes = len(set(labels))
print("Number of clusters: {}".format(num_classes))

# Find biggest class
biggest_class = None
biggest_class_length = 0
for i in range(0, num_classes):
    class_length = len([label for label in labels if label == i])
    if class_length > biggest_class_length:
        biggest_class_length = class_length
        biggest_class = i

print("Biggest cluster id number: {}".format(biggest_class))
print("Number of faces in biggest cluster: {}".format(biggest_class_length))

# Find the indices for the biggest class
indices = []
for i, label in enumerate(labels):
    if label == biggest_class:
        indices.append(i)

print("Indices of images in the biggest cluster: {}".format(str(indices)))

# Ensure output directory exists
if not os.path.isdir(output_folder_path):
    os.makedirs(output_folder_path)

# Save the extracted faces
print("Saving faces in largest cluster to output folder...")
for i, index in enumerate(indices):
    img, shape = images[index]
    file_path = os.path.join(output_folder_path, "face_" + str(i))
    # The size and padding arguments are optional with default size=150x150 and padding=0.25
    dlib.save_face_chip(img, shape, file_path, size=150, padding=0.25)
    

Processing file: F:/PYcode/coding/faceinfo/infomations\001.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\002.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\003.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\004.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\005.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\006.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\007.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\008.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\009.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\010.jpg
Number of faces detected: 1
Processing file: F:/PYcode/coding/faceinfo/infomations\011.jpg
Number of faces detected: 1

In [ ]:
lis = []
for i in descriptors:
#     print(i[0])
    lia = []
    for num in range(len(descriptors[0]) - 1):
        lia.append(i[num]/i[num+1])
        print(lia)
    lis.append([min(lia),max(lia)])